In [37]:
import pandas as pd
import json
import jsonschema as jsh
from jsonschema import validate
import datetime
import pytz
import dateutil.parser as parser
#pytz for time

In [23]:
'''' json.loads loads from a string; load loads from a file ''''

SyntaxError: EOL while scanning string literal (743555468.py, line 1)

In [73]:
file = open('../data/example.json', 'r')
jf = file.readlines()
file.close()

jf

['{"title": "First title", "createdAt": "2020-10-19T02:56:51+0000", "text": "Some post content", "author": "druths", "total_count": "12"}\n',
 '{"title_text": "Second title", "createdAt": "2020-10-18T02:56:51+0000", "text": "Some post content", "author": "druths", "tags": [], "total_count": "12.8"}\n',
 '{"createdAt": "2020-10-19T02:56:51+0000", "text": "Some post content", "author": "druths", "tags": [], "total_count": 19}\n',
 '{"title": "Third title", "createdAt": "2020-10-17T02:56:51+0900", "text": "Some post content", "author": "druths", "total_count": 22, "tags": ["data science", "data", "data annotation"]}\n',
 '{"title_text": "Malformed JSON", "createdAt": "2020-10-17T02:56:51+0700", "text": "Some post content", "author": "druths", "total_count": "12"\n',
 '{"title_text": "Fourth title", "createdAt": "2020-10-17T02:56:51+0700", "text": "Some post content", "author": "druths", "total_count": 22.9, "tags": []}\n',
 '{"title_text": "Fifth title", "createdAt": "12345678", "text": "

In [6]:
schema = {
    "type" : "object",
    "properties" : {
        "title" : {"type" : "string"},
        "title_text" : {"type": "string"},
        "createdAt" : {
            "type" : "string", 
            "format" : "date-time"
        },
        "text" : {"type" : "string"},
        "author" : {"type" : "string"},
        "total_count" : {
            #8. must be int float or string
            "type" : ["number", "string"]
        },
        "tags" : {"type" : "array"}
    },
    #1. requires title ot title_text
    "anyOf" : [
        {"required" : ["title"]},
        {"required" : ["title_text"]}
    ]
}

In [76]:
temp = json.loads(jf[5])
#print(validate(temp, schema = schema))

In [67]:
def check_json(line):
    try:
        temp = json.loads(line)
        validate(temp, schema) 
        return temp
    except Exception as err:
        return -1

In [8]:
#2. rename into title
def rename_title(js):
    try:
        js['title'] = js.pop('title_text')
        return js
    except KeyError as err:
        return js

In [35]:
#3. Standardize createdAt date times to the UTC timezone
#4. remove if date time that can’t be parsed using the ISO datetime standard
#did not check if createdAt is of type date
def stand_date(js):
    try:
        date = js['createdAt']
    except KeyError as err1:
        return js
    
    try:
        date = parser.parse(date)
        js['createdAt'] = date.astimezone(pytz.utc).isoformat()
        return js
    except Exception as err2:
        return -1

In [45]:
#6. Remove all the posts where the author field is empty, null, or N/A.
def check_author(js):
    try:
        author = js['author']
        if author == 'null' or author == 'N/A' or len(author)==0 or author == None:
            return -1
        else:
            return js
    except KeyError as err:
        return js

In [16]:
#7.  total_count: change float and string to int
def check_count(js):
    try:
        count = js['total_count']
    except KeyError as err:
        return js
    
    try:
        js['total_count'] = int(float(count))
        return js
    except ValueError as err2:
        return -1

In [17]:
#9. slice tags[] into individual words
def slice_tag(js):
    try:
        tags = js['tags']
        sentence = " ".join(tags)
        js['tags'] = sentence.split()
        return js
    
    except KeyError as err:
        return js

In [77]:
a = slice_tag(temp)
print(a)

{'title_text': 'Fourth title', 'createdAt': '2020-10-17T02:56:51+0700', 'text': 'Some post content', 'author': 'druths', 'total_count': 22.9, 'tags': []}


In [68]:
my_js = []

for line in jf:
    if check_json(line) == -1:
        continue
    temp = check_json(line)    
    
    if rename_title(temp) == -1:
        continue
    else:
        temp = rename_title(temp)
        
    if stand_date(temp) == -1:
        continue
    else:
        temp = stand_date(temp)
            
    if check_author(temp) == -1:
        continue
    else:
        temp = check_author(temp)
            
    if check_count(temp) == -1:
        continue
    else:
        temp = check_count(temp)
            
    #my_js.append(temp)
    #except JSONDecodeError as err:

        
    temp = slice_tag(temp)
    my_js.append(temp)
    print(temp)
#jf = ast.literal_eval(jf)

#print(my_js)

{'title': 'First title', 'createdAt': '2020-10-19T02:56:51+00:00', 'text': 'Some post content', 'author': 'druths', 'total_count': 12}
{'createdAt': '2020-10-18T02:56:51+00:00', 'text': 'Some post content', 'author': 'druths', 'tags': [], 'total_count': 12, 'title': 'Second title'}
{'title': 'Third title', 'createdAt': '2020-10-16T17:56:51+00:00', 'text': 'Some post content', 'author': 'druths', 'total_count': 22, 'tags': ['data', 'science', 'data', 'data', 'annotation']}


In [56]:
with open("../data/try.json", "w") as output:
    output.truncate(0)
    for dic in my_js:
        json.dump(dic, output)
        output.write("\n")


In [62]:
import unittest
a=['a', 'b', 'c']
b= ['b','c','a']
a.equalTo(b)

AttributeError: 'list' object has no attribute 'equals'